<a href="https://colab.research.google.com/github/Leraniki/Course_hybr_quant_nn/blob/main/course_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение модели

In [3]:
!pip install -q transformers sentencepiece

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Загрузка модели

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:


# 1. Настройки и Данные
model_name = "sberbank-ai/ruRoberta-large"
file_path = 'data.txt'

with open (file_path, 'r', encoding = 'utf-8') as f:
    texts = [line.strip() for line in f.readlines() if len(line.strip()) > 10]
    print(f"Загружено {len(texts)} предложений из файла {file_path}")

# 2. Токенизатор и Датасет
print("Загружаем токенизатор...")
tokenizer = AutoTokenizer.from_pretrained(model_name)


class SimpleTextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=64):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, truncation=True,
                                   padding='max_length')

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)


dataset = SimpleTextDataset(texts, tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
loader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)

# 3. Загрузка ПРЕДОБУЧЕННОЙ модели
print(f"Загружаем веса модели {model_name}...")

model = AutoModelForMaskedLM.from_pretrained(model_name)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Загружено 226 предложений из файла data.txt
Загружаем токенизатор...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Загружаем веса модели sberbank-ai/ruRoberta-large...


### Само обучение

In [4]:
# 4. Цикл Обучения (Fine-tuning)
batch_size = 4
epochs = 5
learning_rate = 2e-5

print(f"Начинаем дообучение на {device}...")

for epoch in range(epochs):
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        with torch.no_grad():
            predictions = torch.argmax(logits, dim=-1)
            active_loss = labels.view(-1) != -100

            flat_preds = predictions.view(-1)[active_loss].cpu().numpy()
            flat_labels = labels.view(-1)[active_loss].cpu().numpy()

            all_preds.extend(flat_preds)
            all_labels.extend(flat_labels)

    avg_loss = total_loss / len(loader)
    acc = accuracy_score(all_labels, all_preds) if all_labels else 0

    print(f"Epoch {epoch + 1}/{epochs} | Loss: {avg_loss:.4f} | Accuracy: {acc:.4f}")

# 5. Демонстрация работы
print("\n--- Тест модели ---")
model.eval()

test_sentence = "Опекуны и [MASK] несовершеннолетних граждан обязаны проживать совместно со своими подопечными. "

inputs = tokenizer(test_sentence, return_tensors="pt").to(device)
with torch.no_grad():
    logits = model(**inputs).logits

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
predicted_word = tokenizer.decode(predicted_token_id)

print(f"Запрос: {test_sentence}")
print(f"Реультат: {predicted_word}")


output_dir = "/content/drive/MyDrive/Курсовая/model"

# 1. Сохраняем и веса, и токенизатор
print(f"Сохраняем модель в папку {output_dir}...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Готово! Модель сохранена.")

Начинаем дообучение на cuda...
Epoch 1/5 | Loss: 1.1131 | Accuracy: 0.7710
Epoch 2/5 | Loss: 1.1287 | Accuracy: 0.7639
Epoch 3/5 | Loss: 0.9206 | Accuracy: 0.7997
Epoch 4/5 | Loss: 0.6547 | Accuracy: 0.8382
Epoch 5/5 | Loss: 0.5660 | Accuracy: 0.8568

--- Тест модели ---
Запрос: Опекуны и [MASK] несовершеннолетних граждан обязаны проживать совместно со своими подопечными. 
Реультат: 
Сохраняем модель в папку /content/drive/MyDrive/Курсовая/model...
Готово! Модель сохранена.


# Тестирование

In [5]:
model_path = "/content/drive/MyDrive/Курсовая/model"

print(f"Загружаю обученную модель из {model_path}...")

try:
    # Загружаем не из интернета, а из твоей папки!
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path)
except OSError:
    print("Ошибка! Не найдена папка с моделью. Укажи правильный путь.")
    exit()

# Если есть видеокарта - используем, нет - и процессор справится быстро
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval() # Переключаем в режим "экзамена" (отключаем обучение)

print("\n=== МОДЕЛЬ ГОТОВА К РАБОТЕ ===")

Загружаю обученную модель из /content/drive/MyDrive/Курсовая/model...

=== МОДЕЛЬ ГОТОВА К РАБОТЕ ===


In [6]:
print(f"Используй токен {tokenizer.mask_token} чтобы скрыть слово.")
print("Пример: Договор вступает в [MASK] с момента подписания.")
print("Для выхода напиши: exit")

while True:
    print("-" * 30)
    user_text = input("Введи предложение: ")

    if user_text.lower() in ['exit', 'quit', 'выход']:
        break

    # Проверка, есть ли маска в тексте
    if tokenizer.mask_token not in user_text:
        # Если пользователь забыл маску, но использовал [MASK], заменим сами (для удобства)
        if "[MASK]" in user_text and tokenizer.mask_token != "[MASK]":
             user_text = user_text.replace("[MASK]", tokenizer.mask_token)
        elif "<mask>" in user_text: # Для RoBERTa
             pass
        else:
            print(f"⚠ Внимание: Ты не добавила маску! Используй {tokenizer.mask_token} или [MASK]")
            continue

    # Подготовка данных
    inputs = tokenizer(user_text, return_tensors="pt").to(device)

    # Предсказание
    with torch.no_grad():
        logits = model(**inputs).logits

    # Находим индекс маски в предложении
    # (Если масок несколько, найдем первую)
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

    if len(mask_token_index) == 0:
        print("Что-то пошло не так, не могу найти маску.")
        continue

    # Берем индекс первой маски
    idx = mask_token_index[0]

    # Находим топ-5 самых вероятных слов
    top_k = 5
    probs = torch.nn.functional.softmax(logits[0, idx], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    print(f"\nМодель думает, что пропущено:")
    for i in range(top_k):
        token_id = top_k_indices[i].item()
        word = tokenizer.decode([token_id]).strip()
        probability = top_k_weights[i].item() * 100
        print(f"{i+1}. {word} ({probability:.1f}%)")

Используй токен <mask> чтобы скрыть слово.
Пример: Договор вступает в [MASK] с момента подписания.
Для выхода напиши: exit
------------------------------
Введи предложение: Опекун не [MASK] без предварительного разрешения органа опеки и попечительства совершать, а попечитель - давать согласие на совершение сделок по отчуждению

Модель думает, что пропущено:
1. вправе (97.7%)
2. может (1.7%)
3. должен (0.4%)
4. обязан (0.1%)
5. права (0.0%)
------------------------------
Введи предложение: Гражданин может быть объявлен судом , если в месте его жительства нет сведений о месте его пребывания в течение пяти лет
⚠ Внимание: Ты не добавила маску! Используй <mask> или [MASK]
------------------------------
Введи предложение: Гражданин может быть объявлен судом <mask>, если в месте его жительства нет сведений о месте его пребывания в течение пяти лет

Модель думает, что пропущено:
1. мертвым (52.5%)
2. больным (16.8%)
3. таковым (15.0%)
4. розыск (4.5%)
5. виновным (1.5%)
----------------------